In [260]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
from arch import arch_model
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

In [261]:
warnings.filterwarnings("ignore")

## Market risk

In [262]:
stocks = {  # stocks and their tickers
    'aapl' : 10 ** 6,
    't'    : 10 ** 6,
    'blk'  : 10 ** 6,
    'msft' : 10 ** 6,
}

def get_adj_close(ticker, start, end):  # get adjusted close price from yahoo finance
    return yf.download(ticker, start=start, end=end)['Adj Close']

def get_portfolio(stocks, start, end):  # get portfolio of stocks
    portfolio = pd.DataFrame()
    for stock in stocks:
        portfolio[stock] = get_adj_close(stocks[stock]['tick'], start, end)
    return portfolio

In [263]:
# expected short fall
def expected_shortfall(returns, alpha, value_at_risk_fn):  # of the time that we loose, how much do we loose
    return returns[returns <= - value_at_risk_fn(returns, alpha)].mean()

In [265]:
# value at risk functions
def non_param_value_at_risk(returns, alpha):  # stupid
    return - returns.quantile(alpha, interpolation='lower')

def non_param_probability_weighted_value_at_risk(returns, alpha, l=0.995):  # time weighted
    returns *= np.array([l ** i for i in range(len(returns))])[:, np.newaxis]
    return - returns.quantile(alpha, interpolation='lower')

def param_value_at_risk(returns, alpha):  # assuming normal distribution
    mu = returns.mean()    # mean
    sigma = returns.std()  # volatility
    return - norm.ppf(alpha, mu, sigma)

def ewma_value_at_risk(returns, lambda_, alpha):  # exponentially weighted moving average
    # this is almost certainly wrong
    return - returns.ewm(alpha=1-lambda_).mean().quantile(alpha, interpolation='lower')

def garch_value_at_risk(returns, alpha):  # generalized autoregressive conditional heteroskedasticity
    # use arch_model function from arch library
    am = arch_model(returns, mean='Zero', vol='GARCH', p=1, o=0, q=1, dist='Normal')
    res = am.fit(disp='off')
    forecasts = res.forecast(horizon=1, start=0)
    return - forecasts.variance.values[-1, :].item() ** 0.5 * norm.ppf(alpha)

In [266]:
portfolio  = get_portfolio(stocks, '2018-01-01', '2019-01-01') if 'prices' not in locals() else eval('prices')
returns    = portfolio.pct_change().dropna()
# expected_shortfall(returns, 0.05, non_param_value_at_risk)
# non_param_probability_weighted_value_at_risk(returns, 0.05)
# param_value_at_risk(returns, 0.05)
# ewma_value_at_risk(returns, 0.94, 0.05)
garch_value_at_risk(returns, 0.05)

ValueError: y must be squeezable to 1 dimension

In [253]:
x = np.random.randn(8)
w = np.array([0.1, 0.2, 0.3, 0.4])
y = np.zeros(x.shape[0])

def moving_average(x, w):
    for i in range(x.shape[0]):
        x_   = x[max(0, i - w.shape[0] + 1): i + 1]
        w_   = w[:x_.shape[0]]
        y[i] = np.sum(x_ * w_) / np.sum(w_)
    return y

# auto regressive


array([1.])

## Derivatives pricing

In [17]:
# geometric brownian motion
def geometric_brownian_motion(S, mu, sigma, W, T, N):
    t = numpy.linspace(0., 1., N + 1)
    S = []
    S.append(S)
    for i in range(1, int(N + 1)):
        drift = (mu - 0.5 * sigma ** 2) * t[i]
        diffusion = sigma * W[i - 1]
        S_temp = S[-1] * numpy.exp(drift + diffusion)
        S.append(S_temp)
    return S, t

In [ ]:
# pricing options
def atm_call_price(S, K, T, r, sigma):
    d1 = (numpy.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * numpy.sqrt(T))
    d2 = d1 - sigma * numpy.sqrt(T)
    return S * norm.cdf(d1) - numpy.exp(-r * T) * K * norm.cdf(d2)

In [14]:
stock = portfolio[stocks['Apple']]

## Delta hedging in the Black-Scholes model

In [18]:
# delta hedge
def delta_hedge(S, K, T, r, sigma, N, M, W):
    dt = T / N
    delta = []
    for i in range(M):
        S_t, t = geometric_brownian_motion(S, r, sigma, W[i], T, N)
        delta.append((atm_call_price(S_t[-1], K, T, r, sigma) - atm_call_price(S_t[-2], K, T, r, sigma)) / (S_t[-1] - S_t[-2]))
    return delta

stock = get_adj_close(stocks['Apple'], '2018-01-01', '2019-01-01')

[*********************100%%**********************]  1 of 1 completed


## Credit Value Adjustment